# Results of LeNet Trained on MNIST

In this notebook I display results in terms of accuracy on test data for LeNet models trained on the MNIST dataset using various regularization techniques. The LeNet model and the MNIST dataset were set up as in Hoffman 2019 (for all the models). The MNIST data is preprocessed by normalizing using mean 0.1307 and variance 0.3081. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization with a dropout rate of p_drop = 0.5. I also train a model with Jacobian Regularization without dropout. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs (150 000 SGD iterations).

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_MNIST
from model_classes import LeNet_MNIST
from tools import accuracy, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_MNIST()

# Summary of model
summary_model = LeNet_MNIST().to(device)
summary(summary_model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
           Dropout-6                  [-1, 120]               0
            Linear-7                   [-1, 84]          10,164
           Dropout-8                   [-1, 84]               0
            Linear-9                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
---------------------------------------------

In [3]:
# Load models
dataset = "mnist"

model_names_set = [
    "model_no_reg",
    "model_no_reg_no_dropout",
    "model_l2",
    "model_jacobi",
    "model_jacobi_no_dropout",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (95%)
0,model_no_reg,98.698 +/- 0.11
1,model_no_reg_no_dropout,98.842 +/- 0.086
2,model_l2,99.166 +/- 0.06
3,model_jacobi,98.66 +/- 0.112
4,model_jacobi_no_dropout,98.892 +/- 0.059
5,model_svb,98.442 +/- 0.058


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 50 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
# model_names = [
#     "model_no_reg_0",
#     "model_l2_0",
#     "model_jacobi_0",
#     "model_jacobi_no_dropout_0",
#     "model_svb_0",
# ]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

  0%|          | 0/5 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './total_variation_mnist_models/model_no_reg_0_total_isotropic_variation.pkl'

In [ ]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                       0.010  \
                                   mean conf_interval          mean   
model_no_reg_0              9851.462817   2997.207366  24154.738848   
model_no_reg_no_dropout_0   4265.181672   1061.030567  10429.056153   
model_l2_0                  2507.904258    588.650567   3988.958497   
model_l2_no_dropout_0       1737.184198    462.547782   4105.181288   
model_jacobi_0             12477.949812    2365.30178  14360.839871   
model_jacobi_no_dropout_0   4986.177974    655.486593    6167.04924   
model_svb_0                 2200.363259    586.473489   3260.420016   
model_svb_no_dropout_0      1399.403256    519.949651    3813.44598   
model_jacobi_and_svb_0      1701.872752    498.231434   2834.387426   
model_all_0                 1874.261099    436.506765   2618.510882   
model_jacobi_and_l2_0       2587.095726    494.336516   1640.370227   
model_svb_and_l2_0          1989.398665    538.141934    3263.88711   

                                                0.001                
                          conf_interval          mean conf_interval  
model_no_reg_0              2830.926312  32063.135764   2738.396433  
model_no_reg_no_dropout_0   1199.697726  31600.211276   2870.781776  
model_l2_0                   466.510796   8839.679244    628.664288  
model_l2_no_dropout_0        441.435794    9459.67566    833.034843  
model_jacobi_0              1915.342674  10440.661813    1262.57879  
model_jacobi_no_dropout_0   1001.032338   3975.907419    840.818795  
model_svb_0                  355.656455   2933.065649    576.805165  
model_svb_no_dropout_0       490.683055   10447.95587   1151.780759  
model_jacobi_and_svb_0       395.224955   7288.245934    802.642919  
model_all_0                  459.703316   3719.760308     424.24716  
model_jacobi_and_l2_0        351.551328    218.302944     75.498574  
model_svb_and_l2_0           464.336097   5283.307119    467.174358

#### Anisotropic Total Variation

In [ ]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
# model_names = [
#     "model_no_reg_0",
#     "model_l2_0",
#     "model_jacobi_0",
#     "model_jacobi_no_dropout_0",
#     "model_svb_0",
# ]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 12/12 [00:00<00:00, 75.45it/s]


In [ ]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                   0.010                \
                               mean conf_interval      mean conf_interval   
model_no_reg_0              13213.5   3549.131109   29129.4   3332.900566   
model_no_reg_no_dropout_0   4215.88   1042.936626  11314.06   1245.593248   
model_l2_0                  3125.58    716.259516    4550.2    554.719388   
model_l2_no_dropout_0       2440.58    540.681529    4790.3     530.38616   
model_jacobi_0             11567.86   2211.607504   14935.9    1994.71354   
model_jacobi_no_dropout_0   5589.24    656.469237   6367.68   1190.711972   
model_svb_0                 2359.96    690.635265    3241.6    427.279752   
model_svb_no_dropout_0      1811.18    577.327569    4095.0     507.40284   
model_jacobi_and_svb_0      1700.58    437.810365   2869.44    386.940485   
model_all_0                 2232.64    592.150483   2787.84    546.191794   
model_jacobi_and_l2_0       2833.14    572.866507    1910.7    410.804639   
model_svb_and_l2_0          2798.66     586.71475    3909.1    501.305186   

                              0.001                
                               mean conf_interval  
model_no_reg_0             39373.74   2579.372207  
model_no_reg_no_dropout_0  35867.64   2946.973752  
model_l2_0                 10357.86   1017.380333  
model_l2_no_dropout_0      10509.82    975.234232  
model_jacobi_0             11271.68   1167.385573  
model_jacobi_no_dropout_0   4999.06   1482.457164  
model_svb_0                 3671.46    702.099279  
model_svb_no_dropout_0     11685.98   1326.283256  
model_jacobi_and_svb_0      8130.78    588.493205  
model_all_0                  3932.5    461.501988  
model_jacobi_and_l2_0         266.6    107.573266  
model_svb_and_l2_0           6236.3    581.986323